In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    AutoConfig,
)

In [3]:
from utils.copied_utils import (
    compute_input_and_target_lengths,
    DataCollatorForT5MLM,
    tokenize_function,
    DataCollatorForNI,
)
import json

In [4]:
before_mask_input_length, target_length = compute_input_and_target_lengths(
    inputs_length=512,
    noise_density=0.15,
    mean_noise_span_length=3.0,
)
before_mask_input_length, target_length

(568, 114)

In [49]:
import torch
import json

class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, filename, tokenizer, before_mask_input_length):
        self.filename = filename
        self.f = open(self.filename)
        self.tokenizer = tokenizer
        self.before_mask_input_length = before_mask_input_length

    def __iter__(self):
        t = []
        while True:
            
            while len(t) < self.before_mask_input_length:
                line = self.f.readline()
                if not line:
                    self.f.close()
                    self.f = open(self.filename)
                    line = self.f.readline()
                try:
                    line = json.loads(line)
                    t_ = self.tokenizer(line)['input_ids']
                    t.extend(t_)
                except BaseException:
                    pass
            
            t_ = t[: self.before_mask_input_length]
            t = t[self.before_mask_input_length:]
            yield {'input_ids': t_}

In [50]:
tokenizer = AutoTokenizer.from_pretrained('./out-small-1.1')

In [51]:
ds = Dataset('combine.jsonl', tokenizer, before_mask_input_length)
i = iter(ds)

In [52]:
config = AutoConfig.from_pretrained(
    './out-small-1.1'
)

In [53]:
data_collator = DataCollatorForT5MLM(
    tokenizer=tokenizer,
    noise_density=0.15,
    mean_noise_span_length=3.0,
    input_length=512,
    target_length=target_length,
    pad_token_id=config.pad_token_id,
)

In [54]:
from torch.utils.data import DataLoader

In [55]:
dl = DataLoader(
    ds,
    collate_fn=data_collator,
    batch_size=10,
    num_workers=2,
    pin_memory=True,
    drop_last=False,
)
i = iter(dl)

"Ask.com dan Photobucket telah memeterai perjanjian dimana Photobucket akan menggunakan enjin carian Ask.com untuk carian imej, video dan juga web yang dilakukan di laman Photobucket. Selain daripada itu, iklan daripada Ask.com juga akan ditunjukkan di laman Photobucket."

secara rasminya. Beberapa fungsi baru telah ditambah dan fungsi sedia ada telah dinaik taraf dalam WordPress 2.6, dan kesemua ini telah dirumuskan oleh pihak WordPress dalam satu video ringkas."

"Sekiranya anda malas untuk menonton video tersebut, berikut merupakan fungsi-fungsi baru yang diperkenalkan didalam WordPress 2.6;"

"Photobucket diperolehi oleh News Corp pada tahun lepas, syarikat sama yang memegang laman MySpace. Walaubagaimanapun, laman MySpace menggunakan enjin carian dan iklan daripada Google. Dalam masa yang sama, Ask.com menggunakan teknologi pengiklanan daripada Google sejak daripada tahun 2004."

"Sama seperti jurublog yang lain, Menteri Penerangan kita juga mempunyai blog beliau tersendiri yang d


"Pengiklan boleh menempah iklan pada mereka dengan kadar RM20 seminggu. Ya, AdtoReal menggunakan tempoh masa seminggu dan bukannya jumlah impression dan sebagainya. Walaubagaimanapun, sekiranya pada sesuatu minggu itu tidak terdapat sebarang pengiklan, maka iklan daripada Google Adsense akan dipusingkan pada waktu itu."
"Berbeza dengan Google Adsense yang menyediakan pengiklanan dalam bentuk teks dan imej, DoubleClick pula menyediakan pengiklanan dalam bentuk imej, video dan flash."


"\nSalah Satu Iklan DoubleClick dalam Nuffnang"
"Untuk menguji prestasi syarikat pengiklan ini, saya telah menghantar blog ini, dan mereka telah approve dan memberikan kod tersebut kepada saya. Nampaknya seperti, para pepamer (publisher) tidak mendapat panel mereka untuk menjaga ruang iklan mereka. Kalau tidak silap, rasanya mereka boleh membuat ruangan publisher pada OpenX. Mungkin kerana ianya masih dalam peringkat beta tertutup, beberapa perkara masih dalam pembangunan. Anda boleh melihat contohnya pa



"Blogger adalah salah satu perkhidmatan blog percuma yang disediakan oleh Pyra Labs pada tahun 1999 dan kemudiannya dibeli ollaman terkemuka seperti Yahoo! dan sebagainya. Di Malaysia sendiri terdapat beberapa orang pembangun laman web menggunakan Drupal. Anda mungkin boleh mencubanya, kerana ianya nampak menarik dan setanding dengan Joomla."
"Blogger Kini Menyokong Bahasa Malaysia"


"BarCampMalaysia : Pengenalan Kepada Drupal"

"Sekiranya anda pernah membaca artikel terdahulu, mungkin anda semua telah mengetahui berkenaan penggunaan YouTube API Chromeless Player oleh Malaysiakini.TV. Hari ini, ketika melayari laman Malaysiakini, sekali lagi saya terperasan perubahan pemain video mereka."
"Sebelum ini, dikatakan bahawa Google dalam perbincangan terakhir untuk membeli Digg dengan harga melebihi $200 juta. Walaubagaimanapun, kini Google dikatakan telah menarik diri daripada membeli Digg."


"Jikalau pemain video tersebut diperbandingkan dengan pemain video YouTube yang biasa, terdapat



"BarCamp Malaysia Diadakah Buat Julung Kalinya"

"Jadi, apakah pendapat anda berkenaan dengan rekabentuk terbaru ini?"
"Artikel ini ditulis sempena dengan BarCampMalaysia yang diadakan selama dua hari, Sabtu (26 Julai) dan Ahad (27 Julai)"


"Mungkin ramai yang sudah mendengar mengenai Google Android. Secara amnya, Android adalah salah satu platform untuk perkakas mudah alih yang mana dibangunkan oleh Google dengan kerjasama Open Handset Alliance."
"CNet Bakal Memperkenalkan Rekabentuk Baru"


"\nSlide Android Pada BarCampMalaysia"

"Google kini telah menutup pendaftaran baru untuk menggunakan Google Page Creator. Page Creator adalah salah satu aplikasi dalam talian yang mana membenarkan anda membangunkan laman tanpa sebarang pengetahuan HTML. Walaupun Google menggunakan rich-text editor yang dibangunkan bersama Page Creator pada beberapa produk yang lain seperti Knol dan sebagainya, ia mengambil keputusan untuk menghentikan pendaftaran baru pada laman tersebut."
"Android bakal dikel


"Walaupun Drupal boleh dinaik-taraf penggunaannya, pemasangan asasnya sudah memadai bagi orang yang tidak mempunyai pengetahuan mengenai pengaturcaraan (programming). Secara asasnya, Drupal menggunakan procedural programming dan bukannya object-oriented programming."
"Internet Explorer 8 Beta 2 Menemui Para Pengguna"


"Ingat lagi mengenai SweetCron yang pernah ditulis di InfoMalaya sebelum ini? Kini anda sudah boleh memuat-turun dan mencubanya secara percuma. Sweetcron ialah sebuah perisian blog automasi yang memuatkan segala aktiviti sosial bagi pengguna. Anda boleh melihat demo penggunaan SweetCron di laman YongFook."
"Drupal kini mempunyai lebih daripada 300,00 pengguna termasuk beberapa yang terbaru yang dipenuhi beberapa ciri-ciri menarik. Beberapa ciri-ciri menarik yang baru yang mana diperkenalkan leh YouTube adalah:"


"Setelah sekian lama Google membeli Jaiku, salah satu servis mobile-blogging sama seperti Twitter, kini Jaiku secara rasminya telah berpindah ke server Google.

...


"Ini hanyalah merupakan langkah pertama Google, dan banyak lagi akan diperkenalkan dalam masa terdekat ini, sebelum memperkenalkan ChromeOS pada tahun hadapan."

"Extension Di Perkenalkan Kepada Saluran Pembangun"

"Setelah sekian lama, kini Facebook membawakan fungsi seakan Twitter, iaitu fungsi untuk menujukan kemaskini status atau membuat pautan ke profail rakan Facebook anda dengan penggunaan simbol \u201c@\u201d. Ianya buat masa ini sedang diperkenalkan kepada semua profail pengguna secara berperingkat."

"\nPenggunaan @ Pada Kemaskini Status"

"Kini, ketika anda mengemaskini status pada Facebook, ketika anda menaip simbol \u201c@\u201d, anda akan diperlihatkan dengan senarai rakan-rakan anda untuk dipilih, dimana anda ingin tujukan mesej tersebut. Anda juga turut boleh menujukan kemaskini status tersebut kepada applikasi, halaman peminat (fan pages), groups dan sebagainya yang terdapat di Facebook."

"Fungsi ini yang serba-sedikit dilihat diambil daripada populariti penggunaanny


"Servis panggilan berasaskan web, Jajah, telah pun memperkenalkan fungsi terbaru mereka, iaitu panggilan Twitter, yang mana membolehkan para pengguna Twitter membuat panggilan percuma sesama mereka dengan mudah, dan yang paling penting, secara percuma."

"Perkhidmatan panggilan melalui Twitter ini masih lagi dalam peringkat beta, dan untuk mula menggunakannya, anda dan pengguna yang anda ingin membuat panggilan itu, perlulah menjadi ahli [email\u00a0protected], selain mempunyai akaun di Twitter. Untuk memulakan panggilan, anda hanya perlu menghantar tweet \u201c@call @namapenggunatwitter\u201d dan ini akan membuatkan komputer rakan anda berbunyi, sekaligus membolehkan anda bersembang secara percuma."

"Sama seperti Twitter yang terkenal dengan 140 hurufnya, [email\u00a0protected] didatangkan dengan tempoh panggilan kurang 2 minit. Selesai sahaja tempoh masa 2 minit, panggilan tersebut akan dimatikan, dan anda perlu membuat panggilan kembali. "

"Sebelum ini, Facebook juga dikatakan ak


"Mungkin ini merupakan langkah pertama Friendster dalam mengintegrasikan Twitter ke dalam laman sosial mereka. Sebelum ini juga, Friendster telah menyertai ribuan pengguna dalam Twitter. Sama-sama kita nantikan perkembangan daripada Friendster dari masa ke semasa."

"Setelah memperkenalkan vanity URL tidak lama dahulu pada Facebook, hari ini, laman sosial terpopular di Malaysia ini memperkenalkan kebolehan untuk log masuk ke Facebook dengan menggunakan kata pengguna. Perkara ini sekaligus menjimatkan masa kerana tidak lagi perlu menaip alamat emel yang panjang lebar."

"\nLog Masuk Facebook Dengan Nama Pengguna"

"Sekiranya sebelum ini anda hanya boleh log masuk ke dalam Facebook dengan menggunakan alamat emel, kini anda sudah boleh log masuk ke dalam Facebook menggunakan nama pengguna anda. Nama pengguna anda di Facebook adalah mengikut alamat URL Facebook anda. Anda masih boleh log masuk ke dalam Facebook menggunakan alamat emel, dan ini hanyalah salah satu cara tambahan untuk log m


"Windows 8 Dan 9 Bakal Membawakan Sokongan 128-bit?"

"Hari ini, Microsoft mengumumkan yang mana mereka bakal melepaskan Office 2010 dengan sebuah versi khas, iaitu Office 2010 Starter. Dan menariknya, Office 2010 Starter yang mana menyediakan fungsi asas aplikasi pejabat dari Microsoft itu bakal diedarkan secara percuma bersama-sama dengan PC baru."

"Tetapi amat mendukacitakan kerana ianya tidak akan diberikan untuk dimuat-turun secara berasingan, sebaliknya diedarkan bersama-sama dengan PC baru yang dibeli bersama-sama sistem operasi Windows. Office 2010 Starter bakal disokong oleh iklan, selain memperkenalkan fungsi yang amat minimum berbanding Office 2010."

"Setelah sekian lama Google Wave diperkenalkan, baru sahaja hari ini saya dapat mencubanya. Bagi yang masih tertanya-tanya, apakah itu Google Wave, ianya merupakan salah satu aplikasi dalam talian yang membolehkan para pengguna berkomunikasi dan berbincang pada masa nyata."

"Jikalau pada Twitter, segala data yang dihantar di


"Googl sentiasa memperkenalkan fungsi-fungsi baru pada produk-produk dan aplikasi web mereka. Terkini, Google mengintegrasikan lagi fungsi Google Docs ke dalam GMail, yang mana melaluinya kini anda boleh melihat preview Google Docs di dalam GMail."

"Sebelum ini, anda perlu membuka pautan tersebut pada tab yang lain, dan Google mendapati ianya tidak digemari oleh para pengguna mereka. Oleh itu, dengan pengenalan Google Docs preview ini, para pengguna boleh melihat dokumen Google Docs pada masa sama melihat emel."

"Fungsi yang sama sepertinya pernah diperkenalkan oleh Google untuk servis-servis yang lain seperti video dari YouTube, gambar dan imej dari Flickr dan Picasa dan sebagainya. Kini, secara automatik, GMail akan mengetahui dokumen Google Docs pada emel, dan seterusnya memperlihatkan nama dan jenis dokumen tersebut, dengan pautan khas untuk melihatnya terlebih dahulu."

"Untuk membolehkan fungsi Google Docs Preview ini, anda hanya perlu ke navigasi Labs yang berada di Settings 

In [74]:
inputs = next(i)
inputs

{'input_ids': tensor([[  423,  2542,  2066,  ...,    15, 32072,     3],
         [  296,   433,   964,  ...,   265, 32072,     3],
         [ 1170,  8139, 17653,  ...,  6886, 32072,     3],
         ...,
         [ 1389, 10259,    16,  ...,  4672, 32072,     3],
         [ 1170,  6661,  2485,  ...,   499, 32072,     3],
         [29960,    15,  1579,  ..., 28934, 32072,     3]]),
 'labels': tensor([[32099, 23448, 24066,  ...,  1762, 10339,     3],
         [32099,  5601,  1085,  ...,  8031,   435,     3],
         [32099,  8628, 32098,  ..., 23327, 22101,     3],
         ...,
         [32099,   433, 11226,  ..., 32072,  8031,     3],
         [32099,  2671, 32098,  ...,  1579,    17,     3],
         [32099,  4852, 32098,  ..., 32072,  1862,     3]])}

In [75]:
tokenizer.decode(inputs['input_ids'][0])

' dihidangkan pada halaman hasil carian, kini Google sudah<extra_id_99> tanpa sebarang gangguan ketika anda ingin membuat carian, selain mengintegr<extra_id_98> fungsi pautan pada Search Suggestion. Menarik bukan?Kredit :<extra_id_97>unchPerkara yang<extra_id_96> diminati pada<extra_id_95> web Google Chrome adalah kepantasan dan betapa mudahnya untuk menggunakan pelayar web tersebut. Kini, melalui kemaskini versi stabil terb<extra_id_94> pelayar webnya lebih 30% laju berbanding sebelumnya.Selain daripada itu, beberapa fungsi yang diperkenalkan pada versi pembangun juga turut diintegrasikan pada versi stabil terbaru ini. Google Chrome 2.0 ini<extra_id_93> pelbagai fungsi termasuk fungsi auto-fill borang, fungsi paparan skrin penuh (full-scre<extra_id_92> kestabilan dan kelajuan pelayar web itu sendiri.\nKadar Kelajuan Membuka Laman Web Yang Makin MeningkatSelain itu, Chrome 2.0 juga membawakan peningkatan kawalan<extra_id_91> pada halaman tab baru. Kini, anda boleh membuang thumbnail la

In [76]:
tokenizer.decode(inputs['input_ids'][1])

' in dan out<extra_id_99>ayari laman web, dan sokongan putaran skrin secara automatik untuk pelayaran web yang lebih menarik dan mudah. HTC Magic yang dijangka bakal berada di pasaran Malaysia mulai 15 Jun ini di<extra_id_98>awangan Blue Cube, akan mula dijual pada harga RM 26<extra_id_97>1999 yang mana memberikan HTC Magic dan juga kad memori mikroSD<extra_id_96>, Maxis membawa masuk iPhone 3G ke dalam Malaysia, dan kini Celcom pula membawa masuk telefon bimbit<extra_id_95> yang dinanti-nantikan oleh para pengguna tempatan. Selain itu, Maxis dan<extra_id_94>com juga turut sama menawarkan Blackberry Storm kepada para penggunanya<extra_id_93>udahkah anda mengemaskini Windows 7 Beta anda<extra_id_92>? Fasa pertama pengakhiran Windows 7 Beta akan bermula dalam<extra_id_91> Julai 2009. Komputer yang menggunakan Windows 7 Beta telah disetkan untuk mula restart setiap 2 jam mulai<extra_id_90> Julai ini.Secara rasminya, Windows 7 Beta<extra_id_89> 1 Ogos 2009, dan para pengguna diingatkan<ext

In [78]:
from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    AutoConfig,
)

In [79]:
config = AutoConfig.from_pretrained('./out-small-1.1')

In [80]:
model = T5ForConditionalGeneration(config)

In [81]:
ls ./logs/2023-07-07/06-17-33/

checkpoint-pt-10000/  checkpoint-pt-20000/  train.log


In [82]:
import torch

model.load_state_dict(torch.load('./logs/2023-07-07/06-17-33/checkpoint-pt-20000/pytorch_model.bin',
                                map_location=torch.device('cpu')))

<All keys matched successfully>

In [83]:
_ = model.eval()

In [84]:
outputs = model(**inputs)

In [87]:
outputs.loss

tensor(2.7687, grad_fn=<NllLossBackward0>)